In [21]:
from alpha_vantage.timeseries import TimeSeries

ts = TimeSeries(
	key="YOUR_API_KEY",
	output_format="pandas",
)

data, meta_data = ts.get_intraday(
	symbol="GOOGL",
	interval="1min",
	outputsize="full",
)

In [22]:
data

,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2024-06-13 19:59:00,175.270,175.370,175.120,175.130,2054.0
2024-06-13 19:58:00,175.250,175.380,175.250,175.270,358.0
2024-06-13 19:57:00,175.240,175.240,175.220,175.240,344.0
2024-06-13 19:56:00,175.250,175.250,175.220,175.220,238.0
2024-06-13 19:55:00,175.250,175.270,175.220,175.270,398.0
...,...,...,...,...,...
2024-05-15 04:04:00,170.434,170.436,170.374,170.375,16.0
2024-05-15 04:03:00,170.364,170.446,170.364,170.444,775.0
2024-05-15 04:02:00,170.314,170.396,170.314,170.395,919.0


In [23]:
meta_data

{'1. Information': 'Intraday (1min) open, high, low, close prices and volume',
 '2. Symbol': 'GOOGL',
 '3. Last Refreshed': '2024-06-13 19:59:00',
 '4. Interval': '1min',
 '5. Output Size': 'Full size',
 '6. Time Zone': 'US/Eastern'}